# Supply Chain Analysis via a Generative AI Agent and a Remote MCP Service

In [31]:
#!pip3 install --upgrade --quiet google-adk neo4j-rust-ext

In [32]:
import os
import random
import sys

In [33]:
import logging

logger = logging.getLogger('agent_neo4j_cypher')
logger.info("Initializing Database for tools")

import logging

logging.getLogger("neo4j").setLevel(logging.INFO)
logging.getLogger("google_genai").setLevel(logging.INFO)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google_genai.types")
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j.notifications")


In [34]:
#env setup
import getpass
import os
from dotenv import load_dotenv

#get env setup
load_dotenv('scp.env', override=True)

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [35]:
from google.adk.models.lite_llm import LiteLlm

OPENAPI_KEY=os.getenv('OPENAPI_KEY')

MODEL = LiteLlm(
    model = "openai/gpt-4.1",
    api_key = OPENAPI_KEY
)

### Connect to Neo4j Database. This is important as a fallback if remote agents don't get invoked

In [36]:
from neo4j import GraphDatabase
from typing import Any
import re

class neo4jDatabase:
    def __init__(self,  neo4j_uri: str, neo4j_username: str, neo4j_password: str):
        """Initialize connection to the Neo4j database"""
        logger.debug(f"Initializing database connection to {neo4j_uri}")
        d = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))
        d.verify_connectivity()
        self.driver = d

    def is_write_query(self, query: str) -> bool:
      return re.search(r"\b(MERGE|CREATE|SET|DELETE|REMOVE|ADD)\b", query, re.IGNORECASE) is not None

    def _execute_query(self, query: str, params: dict[str, Any] | None = None) -> list[dict[str, Any]]:
        """Execute a Cypher query and return results as a list of dictionaries"""
        logger.debug(f"Executing query: {query}")
        try:
            if self.is_write_query(query):
                logger.error(f"Write query not supported {query}")
                raise "Write Queries are not supported in this agent"
                # logger.debug(f"Write query affected {counters}")
                # result = self.driver.execute_query(query, params)
                # counters = vars(result.summary.counters)
                # return [counters]
            else:
                result = self.driver.execute_query(query, params)
                results = [dict(r) for r in result.records]
                logger.debug(f"Read query returned {len(results)} rows")
                return results
        except Exception as e:
            logger.error(f"Database error executing query: {e}\n{query}")
            raise

db = neo4jDatabase(NEO4J_URI,NEO4J_USERNAME,NEO4J_PASSWORD)
db._execute_query("match (n)-[]-() return n limit 2")

ERROR:neo4j.io:[#E175]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('p-e910978e-31de-0011.production-orch-0706.neo4j.io', 7687)) (ResolvedIPv4Address(('35.223.36.122', 7687))): BrokenPipeError(32, 'Broken pipe')
ERROR:neo4j.io:[#E172]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('35.223.36.122', 7687)) (ResolvedIPv4Address(('35.223.36.122', 7687))): BrokenPipeError(32, 'Broken pipe')
ERROR:neo4j.pool:Unable to retrieve routing information


[{'n': <Node element_id='4:c697636e-287e-427a-b5f3-65858f374b15:2557' labels=frozenset({'Suppliers'}) properties={'countryCode': 'US', 'companyName': 'ACME Biologics', 'iso3Code': 'USA'}>},
 {'n': <Node element_id='4:c697636e-287e-427a-b5f3-65858f374b15:61508' labels=frozenset({'Product', 'RM'}) properties={'generation': 'g1', 'productSKU': 'df4be3de-0c6b-4d2c-ac71-db4b9fbdf395', 'package': 'all', 'form': 'Tablet', 'strength': '5mg', 'materialType': 'RM', 'description': 'Iolescidib Tablet 5mg', 'location': 'Philadelphia PA/US', 'globalBrand': 'Iolescidib', 'rmSequence': 1}>}]

### Get basic queries as a fallback - Get Schema and Cypher query execution

In [37]:
def get_schema() -> list[dict[str,Any]]:
  """Get the schema of the database, returns node-types(labels) with their types and attributes and relationships between node-labels
  Args: None
  Returns:
    list[dict[str,Any]]: A list of dictionaries representing the schema of the database
    For example
    ```
    [{'label': 'Person','attributes': {'summary': 'STRING','id': 'STRING unique indexed', 'name': 'STRING indexed'},
      'relationships': {'HAS_PARENT': 'Person', 'HAS_CHILD': 'Person'}}]
    ```
  """
  try:
      results = db._execute_query(
              """
call apoc.meta.data() yield label, property, type, other, unique, index, elementType
where elementType = 'node' and not label starts with '_'
with label,
collect(case when type <> 'RELATIONSHIP' then [property, type + case when unique then " unique" else "" end + case when index then " indexed" else "" end] end) as attributes,
collect(case when type = 'RELATIONSHIP' then [property, head(other)] end) as relationships
RETURN label, apoc.map.fromPairs(attributes) as attributes, apoc.map.fromPairs(relationships) as relationships
              """
          )
      return results
  except Exception as e:
      return [{"error":str(e)}]

In [38]:
#get_schema()

In [39]:
def execute_cypher_query(query: str, params: dict[str, Any]) -> list[dict[str, Any]]:
    """
    Execute a Neo4j Cypher query and return results as a list of dictionaries
    Args:
        query (str): The Cypher query to execute
        params (dict[str, Any], optional): The parameters to pass to the query or None.
    Raises:
        Exception: If there is an error executing the query
    Returns:
        list[dict[str, Any]]: A list of dictionaries representing the query results
    """
    try:
        if params is None:
            params = {}
        results = db._execute_query(query, params)
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [40]:
execute_cypher_query("RETURN 1", None)

[{'1': 1}]

In [41]:
from google.adk.agents import Agent

supplier_agent = Agent(
    model=MODEL,  # e.g. LiteLlm(model="openai/gpt-4")
    name='supplier_agent',
    description="""
    The supply_chain_agent specializes in answering questions related to pharmaceutical supply chain flows,
    raw material sourcing, batch traceability, and distributor demand.
    It uses Cypher queries to retrieve structured insights from the graph, including supplier–API–drug product dependencies,
    bottlenecks, and product lineage.
    Use this agent when the user question involves anything from supplier relationships, product genealogy,
    production stages, or distribution networks.
    """,
    instruction="""
      You are a pharmaceutical supply chain assistant with expertise in Neo4j and Cypher.
      Your job is to trace product flows, map batch genealogy, and assess supply chain risk using graph data.

      - You ALWAYS use the database schema first via the `get_schema` tool and cache it in memory.
      - You generate Cypher queries based on the schema, not just user input — always verify labels and relationship types.
      - For supply path tracing, use a chain like:
        `(:Suppliers)-[:SUPPLIES_RM]-(:RM)-[:PRODUCT_FLOW*]-(:Product)-[:DISTRIBUTED_BY]-(:Distributor)`
      - If asked about single-supplier risks or demand planning, write multi-step queries using aggregation or conditional filters.

      When executing queries, ALWAYS use named parameters (`$sku`, `$brand`, `$market`) and pass them as dictionaries.
      NEVER hardcode values inside the Cypher string — always externalize them into parameters.

      If a Cypher query fails, retry up to 3 times by correcting it using the schema or prior data.
      Use `execute_query` for all data retrieval.
      If results are found, summarize them in natural language and optionally provide table or graph visualization prompts.
      Pass results and control back to parent after completion.
    """,
    tools=[
        get_schema,
        execute_cypher_query
    ]
)

In [42]:
graph_database_agent = Agent(
    model=MODEL,
    name='graph_database_agent',
    description="""
    The graph_database_agent is able to fetch the schema of a neo4j graph database and execute read queries.
    It will generate Cypher queries using the schema to fulfill the information requests and repeatedly
    try to re-create and fix queries that error or don't return the expected results.
    When passing requests to this agent, make sure to have clear specific instructions what data should be retrieved, how,
    if aggregation is required or path expansion.
    Don't use this generic query agent if other, more specific agents are available that can provide the requested information.
    This is meant to be a fallback for structural questions (e.g. number of entities, or aggregation of values or very specific sorting/filtering)
    Or when no other agent provides access to the data (inputs, results and shape) that is needed.
    """,
    instruction="""
      You are an Neo4j graph database and Cypher query expert, that must use the database schema with a user question and repeatedly generate valid cypher statements
      to execute on the database and answer the user's questions in a friendly manner in natural language.
      If in doubt the database schema is always prioritized when it comes to nodes-types (labels) or relationship-types or property names, never take the user's input at face value.
      If the user requests also render tables, charts or other artifacts with the query results.
      Always validate the correct node-labels at the end of a relationship based on the schema.

      If a query fails or doesn't return data, use the error response 3 times to try to fix the generated query and re-run it, don't return the error to the user.
      If you cannot fix the query, explain the issue to the user and apologize.
      *You are prohibited* from using directional arrows (like -> or <-) in the graph patterns, always use undirected patterns like `(:Label)-[:TYPE]-(:Label)`.
      You get negative points for using directional arrays in patterns.

      Fetch the graph database schema first and keep it in session memory to access later for query generation.
      Keep results of previous executions in session memory and access if needed, for instance ids or other attributes of nodes to find them again
      removing the need to ask the user. This also allows for generating shorter, more focused and less error-prone queries
      to for drill downs, sequences and loops.
      If possible resolve names to primary keys or ids and use those for looking up entities.
      The schema always indicates *outgoing* relationship-types from an entity to another entity, the graph patterns read like english language.
      `company has supplier` would be the pattern `(o:Organization)-[:HAS_SUPPLIER]-(s:Organization)`

      To get the schema of a database use the `get_schema` tool without parameters. Store the response of the schema tool in session context
      to access later for query generation.

      To answer a user question generate one or more Cypher statements based on the database schema and the parts of the user question.
      If necessary resolve categorical attributes (like names, countries, industries, publications) first by retrieving them for a set of entities to translate from the user's request.
      Use the `execute_query` tool repeatedly with the Cypher statements, you MUST generate statements that use named query parameters with `$parameter` style names
      and MUST pass them as a second dictionary parameter to the tool, even if empty.
      Parameter data can come from the users requests, prior query results or additional lookup queries.
      After the data for the question has been sufficiently retrieved, pass the data and control back to the parent agent.
    """,
    tools=[
        get_schema, execute_cypher_query
    ]
)

In [43]:
import urllib.parse
import httpx # Make sure httpx is imported at the top of your script

import json
import logging
import asyncio
from typing import Any, Callable, Coroutine, Dict, List
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
#from google.adk.tools.agent_tool import AgentTool

BASE_URL = "https://supply-chain-toolset-373589861902.us-central1.run.app"

POST_TOOLS = [
    "trace_supply_path",
    "dependency_chain",
    "run_cypher",
    "distributors_for_product",
    "logistics_optimization"
]

def create_remote_tool_caller(
    tool_name: str,
    description: str,
    base_url: str
) -> Callable[..., Coroutine[Any, Any, str]]:
    """
    This factory creates an async function to call a remote tool.
    It now uses a much longer timeout to handle slow database queries.
    """
    async def remote_tool_func(**kwargs) -> str:
        tool_endpoint = f"{base_url}/tools/{tool_name}"
        logging.info(f"Preparing to call tool: {tool_name} with args: {kwargs}")
        
        # Set a longer timeout to prevent the client from giving up too early.
        timeout_config = httpx.Timeout(180.0, connect=5.0)

        async with httpx.AsyncClient(timeout=timeout_config) as client:
            try:
                if tool_name in POST_TOOLS:
                    logging.info(f"Calling {tool_name} with POST.")
                    response = await client.post(tool_endpoint, json=kwargs)
                else:
                    if kwargs:
                        query_string = urllib.parse.urlencode(kwargs)
                        full_url = f"{tool_endpoint}?{query_string}"
                        logging.info(f"Calling {tool_name} with GET and query params: {full_url}")
                        response = await client.get(full_url)
                    else:
                        logging.info(f"Calling {tool_name} with GET (no args).")
                        response = await client.get(tool_endpoint)

                response.raise_for_status()
                return json.dumps(response.json())
            
            except httpx.HTTPStatusError as e:
                error_message = f"HTTP Error {e.response.status_code} for {e.request.url}"
                logging.error(f"{error_message}. Server response: {e.response.text}")
                return json.dumps({"error": error_message, "details": e.response.text})
            except httpx.RequestError as e:
                # This will catch timeouts and other connection errors
                error_message = f"A network request error occurred calling {tool_name}: {type(e).__name__}"
                logging.error(error_message)
                return json.dumps({"error": error_message})
    
    remote_tool_func.__name__ = tool_name
    remote_tool_func.__doc__ = description
    return remote_tool_func

## Dynamically Load Tools from the Remote MCP Service

This approach decouples the agent from the tool implementation, allowing us to update the tools on the server without ever changing the agent's code.

In [44]:

async def load_tools_from_remote_server(base_url: str) -> List[FunctionTool]:
    tools_endpoint = f"{base_url}/tools"
    print(f"\nFetching tool definitions from: {tools_endpoint}")
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(tools_endpoint)
            response.raise_for_status()
            tool_definitions = response.json() # This is a LIST
    except httpx.RequestError as e:
        print(f"FATAL: Could not fetch tools from server: {e}")
        return []

    adk_tools = []

     # --- Descriptions are now enhanced based on the Neo4j demo ---
    for tool_info in tool_definitions:
        tool_name = tool_info.get("name")
        if not tool_name: continue

      
        description = tool_info.get("description", "")

        # --- Provide detailed descriptions to guide the agent ---
        if tool_name == "trace_supply_path" or tool_name == "dependency_chain":
            description = (
                "Traces the full supply path for a product, from suppliers to distributors. "
                "Use this for questions about product genealogy, origins, or to see the entire chain. "
                "Requires a 'description' parameter containing the product name. "
                "Example: {'description': 'Nabitegrpultide Caplet 50mg'}"
            )
        elif tool_name == "distributors_for_product":
            description = (
                "Finds all distributors for a specific product. "
                "Requires a 'description' parameter. Example: {'description': 'Nabitegrpultide Caplet 50mg'}"
            )
        elif tool_name == "find_single_supplier_risks":
            description = (
                "Finds raw materials that are only supplied by a single company, highlighting potential risks."
            )
        elif tool_name == "run_cypher":
            description = (
                "A powerful expert tool to execute a custom Cypher query against the database. "
                "Only use this if no other specific tool can answer the user's question. "
                "It REQUIRES a 'query' parameter with the Cypher string. "
                "Example: {'query': 'MATCH (s:Suppliers) RETURN s.companyName LIMIT 5'}"
            )
        
        print(f"  - Tool: {tool_name}")
        
        tool_function = create_remote_tool_caller(
            tool_name=tool_name, description=description, base_url=base_url
        )
        adk_tools.append(FunctionTool(func=tool_function))
       
    
    print(f"Added {len(adk_tools)} tools from MCP service")
    return adk_tools

In [45]:
# equipment_agent = Agent(
#     model=MODEL,
#     name='equipment_agent',
#     description="""
#     Answers questions related to equipment used in pharmaceutical production,
#     including utilization, downtime, maintenance, and equipment-facility mapping.
#     """,
#     instruction="""
#     You are a supply chain equipment analyst.
#     Use Cypher to find which equipment is used where, how often it's maintained, and which products it's associated with.

#     Always check the graph schema for valid node labels like `:Equipment`, `:Facility`, `:Product`, and relationships like `:USED_IN`, `:LOCATED_AT`.
#     Use `get_schema` first, then run queries with `execute_query`.

#     Summarize results clearly, especially if certain equipment shows high downtime or low utilization.
#     """,
#     tools=tools
# )

In [46]:



# batch_trace_agent = Agent(
#     model=MODEL,
#     name='batch_trace_agent',
#     description="""
#     Handles batch genealogy, recall tracing, and contamination mapping.
#     Helps answer questions about where a batch came from and what it touched.
#     """,
#     instruction="""
#     You are an expert in pharmaceutical batch traceability.
#     Use the graph to trace batch origins, production steps, and distribution endpoints.

#     Follow chains like:
#       (:Batch)-[:PART_OF*]->(:Product)-[:DISTRIBUTED_BY]->(:Distributor)

#     Look for shared usage of equipment, suppliers, or ingredients across batches.
#     Use named parameters in Cypher (like `$batchId`, `$productSKU`) and summarize lineage clearly.

#     Use `get_schema` and `execute_query` as needed.
#     """,
#     tools=tools
# )

In [47]:
remote_tools = await load_tools_from_remote_server(BASE_URL)

remote_scp_agent = Agent(
        name="remote_scp_agent",
        description="""Use this agent to access a set of external, remote tools from the MCPServer.
This agent can answer questions about product genealogy, dependency chains, and distributor information.
It is the ONLY agent that can trace supply paths or find distributors.""",
        model=MODEL,
        tools=remote_tools,
        instruction="""You are an expert analyst for a pharmaceutical supply chain graph. Your goal is to answer user questions by selecting the best tool and providing the correct arguments.
Your process MUST be:
1.  Carefully analyze the user's query to understand their core task (e.g., 'trace a path', 'find distributors').
2.  Identify key entities in the query, especially the full product name like 'Nabitegrpultide Caplet 50mg'.
3.  Review your available tools and select the one that is the best semantic match for the user's task.
4.  If the chosen tool requires a 'description' parameter, you MUST use the full product name you identified in step 2 as its value.
5.  Do not call a tool that requires a description without providing one. If you cannot find a product name in the query, you must ask the user for clarification.
"""
    )


Fetching tool definitions from: https://supply-chain-toolset-373589861902.us-central1.run.app/tools
  - Tool: trace_supply_path
  - Tool: dependency_chain
  - Tool: find_single_supplier_risks
  - Tool: top_suppliers_by_product_count
  - Tool: top_suppliers_for_product
  - Tool: raw_materials_by_supplier_count
  - Tool: api_dependency_risk
  - Tool: logistics_optimization
  - Tool: distributors_for_product
  - Tool: run_cypher
  - Tool: get_schema
Added 11 tools from MCP service


### Create expert toolkit from the remote toolset defined. 
### Define Supply Chain Logistics/Optimization, Batch Trace Agent and others

In [48]:

# Toolkit for the batch tracing expert
batch_trace_toolkit = [
    tool for tool in remote_tools
    if tool.func.__name__ in [
        "trace_supply_path",
        "dependency_chain",
        "get_schema"
    ]
]

#Build your specialist agents with their dedicated toolkits
batch_trace_agent = Agent(
        model=MODEL,
        name='batch_trace_agent',
        description="Handles batch genealogy, recall tracing, and contamination mapping.",
        instruction="You are an expert in pharmaceutical batch traceability. Use your tools to trace batch origins and dependencies.",
        tools=batch_trace_toolkit 
    )

In [49]:

logistics_toolkit = [
    tool for tool in remote_tools
    if tool.func.__name__ in [
        "logistics_optimization",
        "get_schema"
    ]
]

#Build your specialist agents with their dedicated toolkits
logistics_optimization_agent = Agent(
        model=MODEL,
        name='logistics_optimization_agent',
        description="Analyzes shipment logistics to find inefficiencies. Use this agent to find cross-border shipment issues or cyclic (looping) delivery routes for a specific product.",
        instruction="""You are an expert in supply chain logistics optimization.
Your primary goal is to use your tools to identify costly and inefficient shipping patterns.
Focus on finding cross-border and cyclic movements in product flows.""",
        tools=logistics_toolkit 
    )

### Supply Chain Root Agent: Contains all remote and local agents as sub agents

In [50]:
supply_chain_root_agent = Agent(
    model=MODEL,
    name='supply_chain_root_agent',
    description="""
    Routes pharmaceutical supply chain questions to the appropriate domain agent.
    Falls back to `database_agent` for schema-level or structural queries.
    """,
    instruction="""
    Route questions to:
    - `supplier_agent` → sourcing, raw materials, supplier risks
    - `equipment_agent` → utilization, downtime, machines
    - `batch_trace_agent` → batch lineage, recalls, genealogy
    - `database_agent` → graph structure, unusual queries, metadata, counts

    Always prefer the most specific agent. Use the database agent only when no other agent fits.
    """,
    sub_agents=[
        remote_scp_agent,
        supplier_agent,
        graph_database_agent,
        batch_trace_agent,
        logistics_optimization_agent,
    ]
)

In [51]:
APP_NAME = 'Neo4j Supply Chain Optimizer'
USER_ID = 'Mr. Neo'

from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent

# Use your actual agent here
runner = InMemoryRunner(app_name=APP_NAME, agent=supply_chain_root_agent)
session = await runner.session_service.create_session(app_name=runner.app_name, user_id=USER_ID)


# Create session
session = await runner.session_service.create_session(
    app_name=runner.app_name,
    user_id=USER_ID
)

# Run prompt
async def run_prompt(new_message: str):
    content = UserContent(parts=[Part(text=new_message)])
    final_response_text = "No response from agent"

    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
                for part in event.content.parts:
                    print(part.text, part.function_call, part.function_response)
            elif event.actions and event.actions.escalate:
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            break

    return final_response_text

In [52]:
await run_prompt('Which raw materials are supplied, give me 100 names and tell how you found out, which agent did you call')

I identified 100 raw materials currently supplied in the supply chain. These were retrieved by querying the graph database for all Raw Material (RM) nodes that are linked from Suppliers nodes via the SUPPLIES_RM relationship. Here are sample names from the results:

- Sulfaipredimultin Tablet 50mg
- Predformin Caplet 5mg
- Rifaiplanin Tablet 100mg
- Nabitegrpultide Caplet 20mg
- Perfluicoxib Tablet 250mg
- Rifadildar Tablet 10mg
- Bolierginicline Caplet 250mg
- Viraaxoapezil Caplet 50mg
- Iolescidib Tablet 10mg
- Predformin Caplet 10mg

...and many more (a total of 100 unique raw material names provided).

How I found this out:
- I used the database schema to understand that raw materials are modeled as RM nodes.
- I checked that the SUPPLIES_RM relationship connects Suppliers to RM.
- I ran a Cypher query to retrieve 100 distinct RM descriptions supplied by any supplier.

You are interacting with the "supplier_agent", as assigned by the supply_chain_root_agent for this domain. If you’

'I identified 100 raw materials currently supplied in the supply chain. These were retrieved by querying the graph database for all Raw Material (RM) nodes that are linked from Suppliers nodes via the SUPPLIES_RM relationship. Here are sample names from the results:\n\n- Sulfaipredimultin Tablet 50mg\n- Predformin Caplet 5mg\n- Rifaiplanin Tablet 100mg\n- Nabitegrpultide Caplet 20mg\n- Perfluicoxib Tablet 250mg\n- Rifadildar Tablet 10mg\n- Bolierginicline Caplet 250mg\n- Viraaxoapezil Caplet 50mg\n- Iolescidib Tablet 10mg\n- Predformin Caplet 10mg\n\n...and many more (a total of 100 unique raw material names provided).\n\nHow I found this out:\n- I used the database schema to understand that raw materials are modeled as RM nodes.\n- I checked that the SUPPLIES_RM relationship connects Suppliers to RM.\n- I ran a Cypher query to retrieve 100 distinct RM descriptions supplied by any supplier.\n\nYou are interacting with the "supplier_agent", as assigned by the supply_chain_root_agent for

In [53]:
await run_prompt("for the above questions, did you call MCPServer toolset ?")

No, I did not call the MCPServer toolset for your previous question.

Here is what happened:
- The supply_chain_root_agent transferred your question directly to me (the supplier_agent).
- As supplier_agent, I accessed the database schema and executed the Cypher query myself, using built-in database tools.
- Specifically, I used the local Cypher query execution tools, not any external or remote MCPServer tools.

If you specifically want information or traces retrieved using the MCPServer toolset (for example, remote genealogy tracing or distributor chains), please let me know and I can transfer your query to the appropriate agent (remote_scp_agent) that uses that toolset. None None


'No, I did not call the MCPServer toolset for your previous question.\n\nHere is what happened:\n- The supply_chain_root_agent transferred your question directly to me (the supplier_agent).\n- As supplier_agent, I accessed the database schema and executed the Cypher query myself, using built-in database tools.\n- Specifically, I used the local Cypher query execution tools, not any external or remote MCPServer tools.\n\nIf you specifically want information or traces retrieved using the MCPServer toolset (for example, remote genealogy tracing or distributor chains), please let me know and I can transfer your query to the appropriate agent (remote_scp_agent) that uses that toolset.'

In [54]:
await run_prompt("can you use the MCPServer toolset to answer the question then?")

Yes, I have now used the MCPServer toolset to answer your question about which raw materials are supplied.

How I did it:
- I used the MCPServer's remote toolset to retrieve a list of raw materials as modeled in the supply chain database.
- Specifically, I used a tool that identifies and lists raw materials, along with their supplier locations and potential supplier risk.
- These results are produced by querying the global supply chain data via MCPServer, rather than querying the local database.

Sample results from the MCPServer toolset include:
- Sulfaipredimultin (Philadelphia PA/US)
- Predformin (Montevideo/UY)
- Rifaiplanin (Montevideo/UY)
- Nabitegrpultide (Lima/PE)
- Perfluicoxib (Montevideo/UY)
- Rifadildar (Montevideo/UY)
- Bolierginicline (Quito/EC)
- Viraaxoapezil (Sao Paulo/BR)
- Iolescidib (Lima/PE)
- Calciiarottecarin (Sao Jose dos Campos/BR)
...and many more (full data contains up to 50, but can be extended).

The agent that answered this is remote_scp_agent, which exclu

"Yes, I have now used the MCPServer toolset to answer your question about which raw materials are supplied.\n\nHow I did it:\n- I used the MCPServer's remote toolset to retrieve a list of raw materials as modeled in the supply chain database.\n- Specifically, I used a tool that identifies and lists raw materials, along with their supplier locations and potential supplier risk.\n- These results are produced by querying the global supply chain data via MCPServer, rather than querying the local database.\n\nSample results from the MCPServer toolset include:\n- Sulfaipredimultin (Philadelphia PA/US)\n- Predformin (Montevideo/UY)\n- Rifaiplanin (Montevideo/UY)\n- Nabitegrpultide (Lima/PE)\n- Perfluicoxib (Montevideo/UY)\n- Rifadildar (Montevideo/UY)\n- Bolierginicline (Quito/EC)\n- Viraaxoapezil (Sao Paulo/BR)\n- Iolescidib (Lima/PE)\n- Calciiarottecarin (Sao Jose dos Campos/BR)\n...and many more (full data contains up to 50, but can be extended).\n\nThe agent that answered this is remote_s

In [55]:
await run_prompt("Trace the supply path for a product Diliprostzolast Tablet 50mg using remote agent")

I attempted to trace the full supply path for the product "Diliprostzolast Tablet 50mg" using the remote agent and the MCPServer toolset. However, no supply path data was returned for this product. This could mean that either the product does not currently have an established supply path in the MCPServer dataset, or that data for this specific product is missing.

If you would like to investigate a different product, or explore related products, please let me know the full product name and I can try tracing the supply path again. None None


'I attempted to trace the full supply path for the product "Diliprostzolast Tablet 50mg" using the remote agent and the MCPServer toolset. However, no supply path data was returned for this product. This could mean that either the product does not currently have an established supply path in the MCPServer dataset, or that data for this specific product is missing.\n\nIf you would like to investigate a different product, or explore related products, please let me know the full product name and I can try tracing the supply path again.'

In [56]:
await run_prompt("List top 25 suppliers with the highest product count?")

Here are the top 25 suppliers with the highest product count, according to the MCPServer remote toolset:

1. SuperPharma Solutions – 17,425 products
2. Major Pharma Labs & BioTech – 14,129
3. Ningbo Nuobai Pharmaceutical – 14,011
4. ExtraPure Biologics – 12,883
5. BioManufacturing Inc – 12,596
6. Guangzhou Pharm & BioLabs – 10,634
7. Chennai BioGenetics – 10,193
8. Serious Genetics – 10,063
9. Regen BioLabs – 9,918
10. Big Pharma – 9,527
11. Star Labs – 9,470
12. BioGenetics Inc – 9,432
13. Yuhan Corporation – 9,344
14. Shanghai BioGenetics – 9,330
15. Chemical INustries – 9,318
16. Nanjing Biologics – 9,318
17. UltraLabs BioGenetics – 9,306
18. Dehli Labs – 9,236
19. Chengdu BioLabs – 9,200
20. BSP Pharmaceuticals S.p.A. – 9,049
21. Ascential Medical & Life Sciences – 8,899
22. Bangalore Biologics – 8,853
23. Recreational BioLabs – 8,562
24. Shanghai Pharmaceuticals – 8,434
25. Another Pharma – 8,402

This ranking was produced by the remote_scp_agent via the MCPServer system, which qu

'Here are the top 25 suppliers with the highest product count, according to the MCPServer remote toolset:\n\n1. SuperPharma Solutions – 17,425 products\n2. Major Pharma Labs & BioTech – 14,129\n3. Ningbo Nuobai Pharmaceutical – 14,011\n4. ExtraPure Biologics – 12,883\n5. BioManufacturing Inc – 12,596\n6. Guangzhou Pharm & BioLabs – 10,634\n7. Chennai BioGenetics – 10,193\n8. Serious Genetics – 10,063\n9. Regen BioLabs – 9,918\n10. Big Pharma – 9,527\n11. Star Labs – 9,470\n12. BioGenetics Inc – 9,432\n13. Yuhan Corporation – 9,344\n14. Shanghai BioGenetics – 9,330\n15. Chemical INustries – 9,318\n16. Nanjing Biologics – 9,318\n17. UltraLabs BioGenetics – 9,306\n18. Dehli Labs – 9,236\n19. Chengdu BioLabs – 9,200\n20. BSP Pharmaceuticals S.p.A. – 9,049\n21. Ascential Medical & Life Sciences – 8,899\n22. Bangalore Biologics – 8,853\n23. Recreational BioLabs – 8,562\n24. Shanghai Pharmaceuticals – 8,434\n25. Another Pharma – 8,402\n\nThis ranking was produced by the remote_scp_agent via t

In [57]:
await run_prompt("Identify APIs used in 5 or more different DPs.")

The following Active Pharmaceutical Ingredients (APIs) are used in 5 or more different Drug Products (DPs), as identified via the MCPServer toolset:

1. Perfluicoxib – used in 6 different DPs
2. Somcoiampa – used in 5 different DPs

This result highlights APIs that are widely shared across multiple drug products, which is relevant for identifying dependency risks and potential supply vulnerabilities. If you need specific product/DPS names associated with each API, let me know! None None


'The following Active Pharmaceutical Ingredients (APIs) are used in 5 or more different Drug Products (DPs), as identified via the MCPServer toolset:\n\n1. Perfluicoxib – used in 6 different DPs\n2. Somcoiampa – used in 5 different DPs\n\nThis result highlights APIs that are widely shared across multiple drug products, which is relevant for identifying dependency risks and potential supply vulnerabilities. If you need specific product/DPS names associated with each API, let me know!'

In [58]:
await run_prompt("Check for cyclic movements in the shipping of Nabitegrpultide?")

Yes, there are cyclic movements identified in the shipping of Nabitegrpultide. According to the logistics analysis via the MCPServer toolset, several supply routes involve the product returning to a previous location before reaching the final distributor—a classic sign of a logistics cycle that can increase costs and delay deliveries.

Noteworthy cyclic movements:
- Nabitegrpultide Caplet 20mg: Raritan NJ/US → San Francisco CA/US → Raritan NJ/US → North America Distributor
- Nabitegrpultide Caplet 50mg: West Point PA/US → Boston MA/US → West Point PA/US → North America Distributor
- Nabitegrpultide Caplet 5mg: St Louis MO/US → Boston MA/US → St Louis MO/US → Canadian Distributor
- Nabitegrpultide Caplet 5mg: Sao Jose dos Campos/BR → Sao Paulo/BR → Sao Jose dos Campos/BR → Panama Distributor

Additionally, there are numerous cross-border shipments with loops, but the above explicitly show a cyclic route.

These cycles were detected by analyzing the geographic shipping paths for "Nabiteg

'Yes, there are cyclic movements identified in the shipping of Nabitegrpultide. According to the logistics analysis via the MCPServer toolset, several supply routes involve the product returning to a previous location before reaching the final distributor—a classic sign of a logistics cycle that can increase costs and delay deliveries.\n\nNoteworthy cyclic movements:\n- Nabitegrpultide Caplet 20mg: Raritan NJ/US → San Francisco CA/US → Raritan NJ/US → North America Distributor\n- Nabitegrpultide Caplet 50mg: West Point PA/US → Boston MA/US → West Point PA/US → North America Distributor\n- Nabitegrpultide Caplet 5mg: St Louis MO/US → Boston MA/US → St Louis MO/US → Canadian Distributor\n- Nabitegrpultide Caplet 5mg: Sao Jose dos Campos/BR → Sao Paulo/BR → Sao Jose dos Campos/BR → Panama Distributor\n\nAdditionally, there are numerous cross-border shipments with loops, but the above explicitly show a cyclic route.\n\nThese cycles were detected by analyzing the geographic shipping paths f

In [59]:
await run_prompt("We are seeing high shipping costs for Calciiarottecarin. Can you find any redundant or looping delivery routes")

Yes, there are multiple redundant or looping (cyclic) delivery routes identified in the shipping of Calciiarottecarin, which likely contribute to the high shipping costs.

Notable cyclic or looping movements include:
- Chicago IL/US → San Francisco CA/US → Chicago IL/US → Canadian Distributor (Caplet, 20mg, g1)
- Collegeville PA/US → Raritan NJ/US → Collegeville PA/US → Canadian Distributor (Caplet, 50mg, g1)
- Boston MA/US → Raritan NJ/US → Boston MA/US → Canadian Distributor (Caplet, 10mg, g2)
- West Point PA/US → Chicago IL/US → West Point PA/US → North America Distributor (Caplet, 20mg & 50mg, g2)
- Raritan NJ/US → Cambridge MA/US → Raritan NJ/US → North America Distributor (Caplet, 5mg, g2)

In addition to these explicit cycles, many routes display repeated cross-border movements where the shipping path returns to an origin city before continuing to the destination distributor, such as:
- Paris/FR → Brussels/BE → Paris/FR → Denmark Distributor
- Sassenheim/NL → Bern/CH → Sassenhei

'Yes, there are multiple redundant or looping (cyclic) delivery routes identified in the shipping of Calciiarottecarin, which likely contribute to the high shipping costs.\n\nNotable cyclic or looping movements include:\n- Chicago IL/US → San Francisco CA/US → Chicago IL/US → Canadian Distributor (Caplet, 20mg, g1)\n- Collegeville PA/US → Raritan NJ/US → Collegeville PA/US → Canadian Distributor (Caplet, 50mg, g1)\n- Boston MA/US → Raritan NJ/US → Boston MA/US → Canadian Distributor (Caplet, 10mg, g2)\n- West Point PA/US → Chicago IL/US → West Point PA/US → North America Distributor (Caplet, 20mg & 50mg, g2)\n- Raritan NJ/US → Cambridge MA/US → Raritan NJ/US → North America Distributor (Caplet, 5mg, g2)\n\nIn addition to these explicit cycles, many routes display repeated cross-border movements where the shipping path returns to an origin city before continuing to the destination distributor, such as:\n- Paris/FR → Brussels/BE → Paris/FR → Denmark Distributor\n- Sassenheim/NL → Bern/CH

In [60]:
await run_prompt("List raw materials and their number of suppliers. Flags ones with single suppliers?")

Here is a list of raw materials along with their locations, and an indicator of supplier risk. All listed raw materials currently have only a single identified supplier (SupplierRisk = 1), which is a flag for potential supply chain vulnerability.

Sample results:
- Sulfaipredimultin (Philadelphia PA/US) — Single supplier
- Predformin (Montevideo/UY) — Single supplier
- Rifaiplanin (Montevideo/UY) — Single supplier
- Nabitegrpultide (Lima/PE) — Single supplier
- Perfluicoxib (Montevideo/UY) — Single supplier
- Rifadildar (Montevideo/UY) — Single supplier
- Bolierginicline (Quito/EC) — Single supplier
- Viraaxoapezil (Sao Paulo/BR) — Single supplier
- Iolescidib (Lima/PE) — Single supplier
- Calciiarottecarin (Sao Jose dos Campos/BR) — Single supplier

...and many more in the dataset, each currently sourced by only one supplier.

Materials with a single supplier are flagged as "SupplierRisk = 1". This is a potential concern for business continuity and risk mitigation—alternative supplier

'Here is a list of raw materials along with their locations, and an indicator of supplier risk. All listed raw materials currently have only a single identified supplier (SupplierRisk = 1), which is a flag for potential supply chain vulnerability.\n\nSample results:\n- Sulfaipredimultin (Philadelphia PA/US) — Single supplier\n- Predformin (Montevideo/UY) — Single supplier\n- Rifaiplanin (Montevideo/UY) — Single supplier\n- Nabitegrpultide (Lima/PE) — Single supplier\n- Perfluicoxib (Montevideo/UY) — Single supplier\n- Rifadildar (Montevideo/UY) — Single supplier\n- Bolierginicline (Quito/EC) — Single supplier\n- Viraaxoapezil (Sao Paulo/BR) — Single supplier\n- Iolescidib (Lima/PE) — Single supplier\n- Calciiarottecarin (Sao Jose dos Campos/BR) — Single supplier\n\n...and many more in the dataset, each currently sourced by only one supplier.\n\nMaterials with a single supplier are flagged as "SupplierRisk = 1". This is a potential concern for business continuity and risk mitigation—alt

In [61]:
sessions_response = await runner.session_service.list_sessions(
    app_name=APP_NAME,
    user_id=USER_ID
)

for session in sessions_response.sessions:
    print(f"Deleting session {session.id}")
    await runner.session_service.delete_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session.id
    )

Deleting session 1863a7a6-dc8b-49f8-8838-a1924e6959cd
Deleting session 708fee3b-4ed3-4516-93ce-cb457abd7254
